# Miranda and Fackler Chap6 の図を再現する

In [1]:
using BasisMatrices
using Plots

## Test multiple interpolation methods

In [7]:
n = 10
a = 0
b = 1
xgrid = collect(linspace(a, b, 101))

101-element Array{Float64,1}:
 0.0 
 0.01
 0.02
 0.03
 0.04
 0.05
 0.06
 0.07
 0.08
 0.09
 0.1 
 0.11
 0.12
 ⋮   
 0.89
 0.9 
 0.91
 0.92
 0.93
 0.94
 0.95
 0.96
 0.97
 0.98
 0.99
 1.0 

### Chebyshev Interp

In [8]:
basis = Basis(ChebParams(n, a, b))
ys = []
m = 9
for i in 1:m
    c = zeros(n)
    c[i] = 1
    y = funeval(c, basis, xgrid)
    push!(ys, y)
end
ylims = (-1.1, 1.1)
plot(xgrid, ys, layout=m, ylims=ylims, leg=false)

### Linear Interp

In [9]:
basis = Basis(LinParams(n, a, b))
ys = []
m = 9
for i in 1:m
    c = zeros(n)
    c[i] = 1
    y = funeval(c, basis, xgrid)
    push!(ys, y)
end
ylims = (-0.1, 1.1)
plot(xgrid, ys, layout=m, ylims=ylims, leg=false)

### Cubic Spline

In [10]:
breaks = [a, b]
evennum = 7
k = 3
m = 9

basis = Basis(SplineParams(breaks, evennum, k))
ys = []
for i in 1:m
    c = zeros(m)
    c[i] = 1
    y = funeval(c, basis, xgrid)
    push!(ys, y)
end
ylims = (-0.1, 1.1)
plot(xgrid, ys, layout=m, ylims=ylims, leg=false)

## Evaluate errors

In [43]:
f1(x) = exp(-x)
f2(x) = 1 ./ (1 + 25x .^ 2)
f3(x) = abs(x).^ 0.5

funcs = [f1, f2, f3]
degrees = [10, 20, 30]

a = -5
b = 5
xgrid = collect(linspace(a, b, 1001))

1001-element Array{Float64,1}:
 -5.0 
 -4.99
 -4.98
 -4.97
 -4.96
 -4.95
 -4.94
 -4.93
 -4.92
 -4.91
 -4.9 
 -4.89
 -4.88
  ⋮   
  4.89
  4.9 
  4.91
  4.92
  4.93
  4.94
  4.95
  4.96
  4.97
  4.98
  4.99
  5.0 

## Table 6.2

### Chebyshev

In [63]:
for (i, f) in enumerate(funcs)
    max_errors = Float64[]
    for d in degrees
        basis = Basis(ChebParams(d, a, b))
        S, = nodes(basis)
        psi = BasisMatrix(basis, Expanded(), S, 0)
        y_true = f(S)
        c = psi.vals[1] \ y_true
        
        y_estimated = funeval(c, basis, xgrid)
        y_true =f(xgrid)
        abs_err = abs(y_true - y_estimated)
        push!(max_errors, maximum(abs_err))
    end
    print("function: f$i, max of absolute errors, width degree 10, 20 and 30: ")
    println(max_errors)
end

function: f1, max of absolute errors, width degree 10, 20 and 30: [0.0141203,1.27073e-10,9.9476e-14]
function: f2, max of absolute errors, width degree 10, 20 and 30: [0.925045,0.747806,0.552433]
function: f3, max of absolute errors, width degree 10, 20 and 30: [0.756759,0.533332,0.435196]


### Linear

In [64]:
for (i, f) in enumerate(funcs)
    max_errors = Float64[]
    for d in degrees
        basis = Basis(LinParams(d, a, b))
        S, = nodes(basis)
        psi = BasisMatrix(basis, Expanded(), S, 0)
        y_true = f(S)
        c = psi.vals[1] \ y_true
        
        y_estimated = funeval(c, basis, xgrid)
        y_true =f(xgrid)
        abs_err = abs(y_true - y_estimated)
        push!(max_errors, maximum(abs_err))
    end
    print("function: f$i, max of absolute errors, width degree 10, 20 and 30: ")
    println(max_errors)
end

function: f1, max of absolute errors, width degree 10, 20 and 30: [13.5956,3.98022,1.86229]
function: f2, max of absolute errors, width degree 10, 20 and 30: [0.885269,0.633874,0.42633]
function: f3, max of absolute errors, width degree 10, 20 and 30: [0.745356,0.512989,0.415227]


### Cubic Spline

In [69]:
breaks = [a, b]
k = 3

for (i, f) in enumerate(funcs)
    max_errors = Float64[]
    for d in degrees
        basis = Basis(SplineParams(breaks, d+1-k, k))
        S, = nodes(basis)
        psi = BasisMatrix(basis, Expanded(), S, 0)
        y_true = f(S)
        c = psi.vals[1] \ y_true
        
        y_estimated = funeval(c, basis, xgrid)
        y_true =f(xgrid)
        abs_err = abs(y_true - y_estimated)
        push!(max_errors, maximum(abs_err))
    end
    print("function: f$i, max of absolute errors, width degree 10, 20 and 30: ")
    println(max_errors)
end

function: f1, max of absolute errors, width degree 10, 20 and 30: [0.357374,0.0231122,0.00511311]
function: f2, max of absolute errors, width degree 10, 20 and 30: [0.914668,0.631624,0.379954]
function: f3, max of absolute errors, width degree 10, 20 and 30: [0.739583,0.474655,0.376635]
